# Group Project

## Data Preprocess

**This is to preprocess the data scraped from the website and create metadata csv file. One of the result is a csv file containing lists to diaplay tags or articles at the same webpage, which is convinient for human to read. Another one dosen't contain lists, where all the content exist just as strings.This is suitable for computational analyzing.**

by:zhuoli Lu




## Import Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Installing and Importing

In [2]:
# Install and import spacy and plotly.
%pip install spaCy
%pip install plotly
%pip install nbformat --upgrade

In [16]:
# Import spacy
import spacy

# Import os to upload documents and metadata
import os

import yaml

# Import json
import json

# Load spaCy visualizer
from spacy import displacy

# Import pandas DataFrame packages
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

# Import graphing package
import plotly.graph_objects as go
import plotly.express as px

## Define a function to extract JSON onjects

In [25]:
def extract_json_objects(text, delimiter=','):

    # Create an empty list to store individual JSON objects,track the depth of braces and starting position of a JSON object
    objs = []
    depth = 0
    start = 0
    # Iterate over each character in the text
    for i, char in enumerate(text):
        if char == '{':
            if depth == 0:
                start = i  # Start of a new JSON object
            depth += 1 # Increase depth for each opening brace
        elif char == '}':
            depth -= 1   # Decrease depth for each closing brace
            if depth == 0:
                # to find a complete JSON object
                try:
                # Try to parse the JSON object and add it to the list
                    objs.append(json.loads(text[start:i+1]))
                except json.JSONDecodeError as e:
                  # Print an error message if JSON is not properly formatted
                    print(f"Error parsing JSON: {e}")
        elif char == delimiter and depth == 0:
           # Ignore the top-level separator (e.g., comma between JSON objects)
            continue
    # Return the list of parsed JSON objects
    return objs


## Create a DataFrame from parsing JSON strings in all files

In [26]:


# Create empty list for storing data
all_data = []

# Specify the directory where the files are located
folder_path = '/content/drive/MyDrive/Guardian_corpus_txt'

# Iterate through each file in the folder
for file_name in os.listdir(folder_path):
    # Construct the full file path
    file_path = os.path.join(folder_path, file_name)

    # Make sure to process txt files only
    if os.path.isfile(file_path) and file_path.endswith('.txt'):
        # Read txt files
        with open(file_path, 'r', encoding='utf-8') as file:
          file_content = file.read()

        # Use yaml to parse JSON data
        try:
            data = yaml.safe_load(file_content)
            all_data.append(data)
        except yaml.YAMLError as e:
            print(f"Error parsing JSON in {file_name}: {e}")

# Transform the data into Pandas DataFrame
df = pd.DataFrame(all_data)

# Look at the head of the DataFrame
print(f"Total files processed: {len(all_data)}")
df.head()


Total files processed: 1220


,type,webTitle,sectionName,webPublicationDate,webUrl,tags,tagCount,elements
0,article,Israel’s Knesset elects Netanyahu ally tempora...,World news,2022-12-13T10:05:04Z,https://www.theguardian.com/world/2022/dec/13/...,"[{'tagTitle': 'Israel', 'tagURL': 'https://www...",7,"[{'id': '63971be08f080e98882a4e50', 'bodyTextS..."
1,article,Out-of-this-world revelations in short supply ...,World news,2022-05-17T17:28:18Z,https://www.theguardian.com/world/2022/may/17/...,"[{'tagTitle': 'UFOs', 'tagURL': 'https://www.t...",11,"[{'id': '6283dd5c8f08261baccdbd5b', 'bodyTextS..."
2,article,Russia-Ukraine war at a glance: what we know o...,World news,2022-12-24T02:29:38Z,https://www.theguardian.com/world/2022/dec/24/...,[{'tagTitle': 'Russia-Ukraine war at a glance'...,10,"[{'id': '63a64a8f8f08c23d9f5d6f88', 'bodyTextS..."
3,article,US Indigenous women face high rates of sexual ...,World news,2022-05-17T12:00:06Z,https://www.theguardian.com/world/2022/may/17/...,"[{'tagTitle': 'Indigenous peoples', 'tagURL': ...",10,"[{'id': '6283baab8f08261baccdbb99', 'bodyTextS..."
4,article,Angela Merkel says she lost influence over Put...,World news,2022-11-25T18:58:26Z,https://www.theguardian.com/world/2022/nov/25/...,"[{'tagTitle': 'Angela Merkel', 'tagURL': 'http...",14,"[{'id': '636918f38f086bfc76546277', 'bodyTextS..."


In [68]:
import pandas as pd

# Assuming you have an original DataFrame named df
articles = {}

# Iterate through the original DataFrame
for index, row in df.iterrows():
    # Extract the title and publication time
    title = row['webTitle']  # Assuming the article title is in the 'webTitle' column
    time = row['webPublicationDate']

    # Initialize the article in the dictionary if it's new
    if title not in articles:
        articles[title] = {'PubTime': time, 'Text': []}

    # Check if the current element contains 'bodyTextSummary'
    if isinstance(row['elements'], list):
        for element in row['elements']:
            if 'bodyTextSummary' in element:
                # Extract the 'bodyTextSummary' and append it to the article's list
                summary = element['bodyTextSummary']
                articles[title]['Text'].append(summary)

# Now create lists for the new DataFrame
web_titles = []
body_text_summaries = []

# Extract data from the articles dictionary
for title, info in articles.items():
    web_titles.append(title)
    body_text_summaries.append(info['Text'])  # This is now a list of all summaries for the article

# Create a new DataFrame
d1 = {'WebTitle': web_titles, 'Text': body_text_summaries}
title_textlist_df = pd.DataFrame(d1)

# Display the first few rows of the new DataFrame
title_textlist_df.head()


,WebTitle,Text
0,Israel’s Knesset elects Netanyahu ally tempora...,[Benjamin Netanyahu’s Likud party has elected ...
1,Out-of-this-world revelations in short supply ...,[A China Eastern Airlines plane that crashed i...
2,Russia-Ukraine war at a glance: what we know o...,[Vladimir Putin has ordered Russia’s defence i...
3,US Indigenous women face high rates of sexual ...,"[The new French prime minister, Élisabeth Born..."
4,Angela Merkel says she lost influence over Put...,[Jamie Forsstroem is excited. Tomorrow they ma...


## Extracting titles, time, tags, url, putting tags at the same page in one list. Creating a dataframe.

In [85]:
import pandas as pd

# Assuming you have an original DataFrame named df
web_titles = []
pub_time = []
web_tags = []  # This will store lists of tags for each article
tag_count = []
web_url = []

# Iterate through the original DataFrame
for index, row in df.iterrows():
    # Extract the title, publication time, and tag count
    title = row['webTitle']  # Assuming the article title is in the 'webTitle' column
    time = row['webPublicationDate']
    count = row['tagCount']
    url = row['webUrl']

    # Collect all tags for this article
    tags = [tag['tagTitle'] for tag in row['tags'] if 'tagTitle' in tag] if isinstance(row['tags'], list) else []

    # Append the information to the lists
    web_titles.append(title)
    pub_time.append(time)
    tag_count.append(count)
    web_tags.append(tags)  # Add the list of tags directly
    web_url.append(url)

# Create a new DataFrame
d2 = {'WebTitle': web_titles, 'WebUrl' : web_url, 'PubTime': pub_time, 'Tags': web_tags, 'TagCounts': tag_count}
title_taglist_df = pd.DataFrame(d2)

# Display the first few rows of the new DataFrame
title_taglist_df.head()

,WebTitle,WebUrl,PubTime,Tags,TagCounts
0,Israel’s Knesset elects Netanyahu ally tempora...,https://www.theguardian.com/world/2022/dec/13/...,2022-12-13T10:05:04Z,"[Israel, Benjamin Netanyahu, Middle East and n...",7
1,Out-of-this-world revelations in short supply ...,https://www.theguardian.com/world/2022/may/17/...,2022-05-17T17:28:18Z,"[UFOs, US military, US news, World news, Artic...",11
2,Russia-Ukraine war at a glance: what we know o...,https://www.theguardian.com/world/2022/dec/24/...,2022-12-24T02:29:38Z,"[Russia-Ukraine war at a glance, Ukraine, Russ...",10
3,US Indigenous women face high rates of sexual ...,https://www.theguardian.com/world/2022/may/17/...,2022-05-17T12:00:06Z,"[Indigenous peoples, US policing, Violence aga...",10
4,Angela Merkel says she lost influence over Put...,https://www.theguardian.com/world/2022/nov/25/...,2022-11-25T18:58:26Z,"[Angela Merkel, Germany, Europe, This is Europ...",14


## Merging the two dataframe above, creating a metadata csv file. Tags and articles in the same webpage exist as lists.

In [91]:
metadata_list_df = title_taglist_df.merge(title_textlist_df,on='WebTitle')


## Transforming the time format to a datatime format, which is readable for machine in python

In [92]:
metadata_list_df['PubTime'] = pd.to_datetime(data_list_df['PubTime'])

metadata_list_df.head()

,WebTitle,WebUrl,PubTime,Tags,TagCounts,Text
0,Israel’s Knesset elects Netanyahu ally tempora...,https://www.theguardian.com/world/2022/dec/13/...,2022-12-13 10:05:04+00:00,"[Israel, Benjamin Netanyahu, Middle East and n...",7,[Benjamin Netanyahu’s Likud party has elected ...
1,Out-of-this-world revelations in short supply ...,https://www.theguardian.com/world/2022/may/17/...,2022-05-17 17:28:18+00:00,"[UFOs, US military, US news, World news, Artic...",11,[A China Eastern Airlines plane that crashed i...
2,Russia-Ukraine war at a glance: what we know o...,https://www.theguardian.com/world/2022/dec/24/...,2022-12-24 02:29:38+00:00,"[Russia-Ukraine war at a glance, Ukraine, Russ...",10,[Vladimir Putin has ordered Russia’s defence i...
3,US Indigenous women face high rates of sexual ...,https://www.theguardian.com/world/2022/may/17/...,2022-05-17 12:00:06+00:00,"[Indigenous peoples, US policing, Violence aga...",10,"[The new French prime minister, Élisabeth Born..."
4,Angela Merkel says she lost influence over Put...,https://www.theguardian.com/world/2022/nov/25/...,2022-11-25 18:58:26+00:00,"[Angela Merkel, Germany, Europe, This is Europ...",14,[Jamie Forsstroem is excited. Tomorrow they ma...


## Parsing every lements, extracting all the articles, putting articles in the same webpage in a row, and spliting them by ",". Creating a dataframe.

In [78]:
# Assuming you have an original DataFrame named original_df
# Used to store extracted data
web_titles = []
pub_time = []
web_tags = []
tag_count = []
web_url = []


# Iterate through the original DataFrame
for index, row in df.iterrows():
    # Extract the title
    title = row['webTitle']  # Assuming the article title is in the 'webTitle' column
    time = row ['webPublicationDate']
    count = row['tagCount']
    url = row['webUrl']


    temp_tags = []



    # Examine whether the web has been processed
    if len(web_titles) > 0 and title == web_titles[-1] and time == pub_time[-1]:
        # if yes, add the new tag as the last tag in one raw
        if isinstance(row['tags'], list):
            for tag in row['tags']:
                if 'tagTitle' in tag:
                    web_tags[-1] += ", " + tag['tagTitle']  # Append the tag to the last item
    else:
        # if else, then process it as a new web in a new raw
        temp_tags = [tag['tagTitle'] for tag in row['tags'] if 'tagTitle' in tag] if isinstance(row['tags'], list) else []
        web_titles.append(title)
        pub_time.append(time)
        tag_count.append(count)
        web_url.append(url)
        web_tags.append(', '.join(temp_tags))  # Convert the list of tags to a comma-separated string


# Create a new DataFrame
d3 = {'WebTitle': web_titles, 'WebUrl' : web_url, 'PubTime':pub_time, 'Tags':web_tags, 'TagCounts':tag_count}
title_tag_df = pd.DataFrame(d3)

# Display the first few rows of the new DataFrame
title_tag_df.head()

,WebTitle,WebUrl,PubTime,Tags,TagCounts
0,Israel’s Knesset elects Netanyahu ally tempora...,https://www.theguardian.com/world/2022/dec/13/...,2022-12-13T10:05:04Z,"Israel, Benjamin Netanyahu, Middle East and no...",7
1,Out-of-this-world revelations in short supply ...,https://www.theguardian.com/world/2022/may/17/...,2022-05-17T17:28:18Z,"UFOs, US military, US news, World news, Articl...",11
2,Russia-Ukraine war at a glance: what we know o...,https://www.theguardian.com/world/2022/dec/24/...,2022-12-24T02:29:38Z,"Russia-Ukraine war at a glance, Ukraine, Russi...",10
3,US Indigenous women face high rates of sexual ...,https://www.theguardian.com/world/2022/may/17/...,2022-05-17T12:00:06Z,"Indigenous peoples, US policing, Violence agai...",10
4,Angela Merkel says she lost influence over Put...,https://www.theguardian.com/world/2022/nov/25/...,2022-11-25T18:58:26Z,"Angela Merkel, Germany, Europe, This is Europe...",14


## Extracting titles, time, tags, url, putting tags at the same webpage in a row, and spliting them by " ". Creating a dataframe.

In [103]:

# Assuming you have an original DataFrame named df
web_titles = []
body_text_summaries = []  # This will now store a string of summaries for each article

# Iterate through the original DataFrame
for index, row in df.iterrows():
    # Extract the title and publication time
    title = row['webTitle']  # Assuming the article title is in the 'webTitle' column
    time = row['webPublicationDate']  # Assuming the publication time is in the 'webPublicationDate' column

    # Collect all summaries for this article
    summaries = [element['bodyTextSummary'] for element in row['elements'] if 'bodyTextSummary' in element] if isinstance(row['elements'], list) else []

    # Check if this title and time has already been added
    if len(web_titles) > 0 and title == web_titles[-1] and time == pub_time[-1]:
        # Append additional summaries to the last entry
        body_text_summaries[-1] += " " + " ".join(summaries)
    else:
        # New article, add all info
        web_titles.append(title)
        body_text_summaries.append(", ".join(summaries))  # Convert the list of summaries to a comma-separated string

# Create a new DataFrame
d4 = {'WebTitle': web_titles, 'Text': body_text_summaries}
title_text_df = pd.DataFrame(d4)

# Display the first few rows of the new DataFrame
title_text_df.head()

,WebTitle,Text
0,Israel’s Knesset elects Netanyahu ally tempora...,Benjamin Netanyahu’s Likud party has elected o...
1,Out-of-this-world revelations in short supply ...,A China Eastern Airlines plane that crashed in...
2,Russia-Ukraine war at a glance: what we know o...,Vladimir Putin has ordered Russia’s defence in...
3,US Indigenous women face high rates of sexual ...,"The new French prime minister, Élisabeth Borne..."
4,Angela Merkel says she lost influence over Put...,Jamie Forsstroem is excited. Tomorrow they mak...


## Merging the two dataframe above, creating a metadata csv file. Tags and articles in the same webpage exist in one row, splited by ",".

In [104]:
metadata_df = title_tag_df.merge(title_text_df,on='WebTitle')


## Transforming the time format to a datatime format, which is readable for machine in python

In [111]:
metadata_df['PubTime'] = pd.to_datetime(metadata_df['PubTime'])

metadata_df.head()

,WebTitle,WebUrl,PubTime,Tags,TagCounts,Text
0,Israel’s Knesset elects Netanyahu ally tempora...,https://www.theguardian.com/world/2022/dec/13/...,2022-12-13 10:05:04+00:00,"Israel, Benjamin Netanyahu, Middle East and no...",7,Benjamin Netanyahu’s Likud party has elected o...
1,Out-of-this-world revelations in short supply ...,https://www.theguardian.com/world/2022/may/17/...,2022-05-17 17:28:18+00:00,"UFOs, US military, US news, World news, Articl...",11,A China Eastern Airlines plane that crashed in...
2,Russia-Ukraine war at a glance: what we know o...,https://www.theguardian.com/world/2022/dec/24/...,2022-12-24 02:29:38+00:00,"Russia-Ukraine war at a glance, Ukraine, Russi...",10,Vladimir Putin has ordered Russia’s defence in...
3,US Indigenous women face high rates of sexual ...,https://www.theguardian.com/world/2022/may/17/...,2022-05-17 12:00:06+00:00,"Indigenous peoples, US policing, Violence agai...",10,"The new French prime minister, Élisabeth Borne..."
4,Angela Merkel says she lost influence over Put...,https://www.theguardian.com/world/2022/nov/25/...,2022-11-25 18:58:26+00:00,"Angela Merkel, Germany, Europe, This is Europe...",14,Jamie Forsstroem is excited. Tomorrow they mak...


## The beginnings of some of the texts may contain extra spaces (indicated by \t or \n). These characters can be replaced by a single space using the str.replace() method.

In [112]:
# Remove extra spaces from papers
metadata_df['Text'] = metadata_df['Text'].str.replace('\s+', ' ', regex=True).str.strip()
metadata_df.head()


,WebTitle,WebUrl,PubTime,Tags,TagCounts,Text
0,Israel’s Knesset elects Netanyahu ally tempora...,https://www.theguardian.com/world/2022/dec/13/...,2022-12-13 10:05:04+00:00,"Israel, Benjamin Netanyahu, Middle East and no...",7,Benjamin Netanyahu’s Likud party has elected o...
1,Out-of-this-world revelations in short supply ...,https://www.theguardian.com/world/2022/may/17/...,2022-05-17 17:28:18+00:00,"UFOs, US military, US news, World news, Articl...",11,A China Eastern Airlines plane that crashed in...
2,Russia-Ukraine war at a glance: what we know o...,https://www.theguardian.com/world/2022/dec/24/...,2022-12-24 02:29:38+00:00,"Russia-Ukraine war at a glance, Ukraine, Russi...",10,Vladimir Putin has ordered Russia’s defence in...
3,US Indigenous women face high rates of sexual ...,https://www.theguardian.com/world/2022/may/17/...,2022-05-17 12:00:06+00:00,"Indigenous peoples, US policing, Violence agai...",10,"The new French prime minister, Élisabeth Borne..."
4,Angela Merkel says she lost influence over Put...,https://www.theguardian.com/world/2022/nov/25/...,2022-11-25 18:58:26+00:00,"Angela Merkel, Germany, Europe, This is Europe...",14,Jamie Forsstroem is excited. Tomorrow they mak...


In [113]:

metadata_list_df['Text'] = metadata_list_df['Text'].apply(lambda x: [' '.join(item.split()).strip() for item in x])
metadata_list_df.head()

,WebTitle,WebUrl,PubTime,Tags,TagCounts,Text
0,Israel’s Knesset elects Netanyahu ally tempora...,https://www.theguardian.com/world/2022/dec/13/...,2022-12-13 10:05:04+00:00,"[Israel, Benjamin Netanyahu, Middle East and n...",7,[Benjamin Netanyahu’s Likud party has elected ...
1,Out-of-this-world revelations in short supply ...,https://www.theguardian.com/world/2022/may/17/...,2022-05-17 17:28:18+00:00,"[UFOs, US military, US news, World news, Artic...",11,[A China Eastern Airlines plane that crashed i...
2,Russia-Ukraine war at a glance: what we know o...,https://www.theguardian.com/world/2022/dec/24/...,2022-12-24 02:29:38+00:00,"[Russia-Ukraine war at a glance, Ukraine, Russ...",10,[Vladimir Putin has ordered Russia’s defence i...
3,US Indigenous women face high rates of sexual ...,https://www.theguardian.com/world/2022/may/17/...,2022-05-17 12:00:06+00:00,"[Indigenous peoples, US policing, Violence aga...",10,"[The new French prime minister, Élisabeth Born..."
4,Angela Merkel says she lost influence over Put...,https://www.theguardian.com/world/2022/nov/25/...,2022-11-25 18:58:26+00:00,"[Angela Merkel, Germany, Europe, This is Europ...",14,[Jamie Forsstroem is excited. Tomorrow they ma...


## Saving the metadata csv files.

In [114]:
metadata_list_df.to_csv('metadata_list.csv')
metadata_df.to_csv('metadata.csv')